In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_boston
boston = load_boston()

bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
print(bos.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  
0     15.3  396.90   4.98  
1     17.8  396.90   9.14  
2     17.8  392.83   4.03  
3     18.7  394.63   2.94  
4     18.7  396.90   5.33  


In [2]:
bos["PRICE"] = boston.target
X = bos.drop("PRICE", axis = 1) # "Price" değişkeni hariç bütün değişkenler girdi değişkeni olarak atandı.
Y = bos["PRICE"] #"Price" değişkeni çıktı değişkeni olarak atandı.

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [4]:
from sklearn.preprocessing import StandardScaler  #Verinin normalize edilmesi
sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [5]:
#Çıktı değişkeni sürekli ise Regresyon kullanılır.
# 1- DOĞRUSAL REGRESYON

In [6]:
from sklearn.linear_model import LinearRegression
#Normalize edilmiş veri ile çalışılır. (std)
linreg = LinearRegression().fit(x_train_std, y_train) #Model tanımlandı ve öğrenme süreci(fit) gerçekleştirildi.
#Modelin denklemini yazdırmak.
print("Doğrusal model sabit: {}" .format(linreg.intercept_))# Kullanılan _ işareti eğitim verisinin kullanılacağını belirtir.
print("Doğrusal model katsayılar (w): {}" .format(linreg.coef_))
#Performans ölçümü
print("R-kare (eğitim): {:.3f}" .format(linreg.score(x_train_std, y_train.values.ravel())))
print("R-kare (test): {:.3f}" .format(linreg.score(x_test_std, y_test.values.ravel())))

Doğrusal model sabit: 22.7454802259887
Doğrusal model katsayılar (w): [-1.01190059  1.05028027  0.07920966  0.618962   -1.87369102  2.70526979
 -0.27957264 -3.09766486  2.09689998 -1.88606339 -2.26110466  0.58264309
 -3.44049838]
R-kare (eğitim): 0.765
R-kare (test): 0.673


In [7]:
#1.2- RİDGE REGRESYON  (minimizasyon modelidir) (L2 cezalandırması ile yapılan regresyon, katsayıların kareleri ceza fonksiyonuna eklenir, Amaç büyük katsayıların etkisini azaltmak veya küçültmek.)
from sklearn.linear_model import Ridge
linridge = Ridge(alpha = 1).fit(x_train_std, y_train)
#Normalize edilmiş veri ile çalışılır. (std)
print("Ridge regresyon sabit: {}" .format(linridge.intercept_))
print("Ridge regresyon katsayılar:\n {}" .format(linridge.coef_))
#Performans ölçümü
print("R-kare (eğitim): {:.3f}" .format(linridge.score(x_train_std, y_train.values.ravel())))
print("R-kare (test): {:.3f}" .format(linridge.score(x_test_std, y_test.values.ravel())))
print("0 olmayan öznitelik(katsayı) sayısı: {}" .format(np.sum(linridge.coef_ != 0))) #Bu bilgi L2 yani Ridge regresyonunda bir etkisi yok , Lasso yani L1 regresyonunda etkilidir.


Ridge regresyon sabit: 22.7454802259887
Ridge regresyon katsayılar:
 [-1.00200663  1.03147899  0.04980509  0.62374999 -1.83526275  2.71572801
 -0.28544778 -3.05889965  2.01159113 -1.80651084 -2.25197658  0.58293035
 -3.4245575 ]
R-kare (eğitim): 0.765
R-kare (test): 0.673
0 olmayan öznitelik(katsayı) sayısı: 13


In [8]:
#Normal regresyon modelinde aşırı öğrenme olmadığı için cezalandırma regresyonun bir faydası olmadı.

In [9]:
# 1.3- LASSO (L1) REGRESYON
#Katsayıların mutlak değerlerini modele katar, bazı katsayıları 0'lar ve denklemden çıkarır, modelin karmaşıklığını azaltmak için kullanılır.
from sklearn.linear_model import Lasso
linlasso = Lasso(max_iter = 20000).fit(x_train_std, y_train)
#Normalize edilmiş veri ile çalışılır. (std)
print("Lasso regresyon sabit: {}" .format(linlasso.intercept_))
print("Lasso regresyon ağırlıklar:\n {}" .format(linlasso.coef_))
print("0 olmayan özniteliklerin sayısı: {}" .format(np.sum(linlasso.coef_ != 0)))
print("R-kare eğitim: {:.3f}" .format(linlasso.score(x_train_std, y_train.values.ravel())))
print("R-kare test: {:.3f}" .format(linlasso.score(x_test_std, y_test.values.ravel())))
print("0 olmayan özniteliklerin ağırlıkları(mutlak değer sırasında):")

for e in sorted (list(zip(list(X), linlasso.coef_)), 
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print("\t{}, {:.3f}" .format(e[0], e[1]))

Lasso regresyon sabit: 22.7454802259887
Lasso regresyon ağırlıklar:
 [-3.40902715e-02  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  2.67547491e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -1.19742663e-01 -1.78483588e+00  2.43989358e-03
 -3.40428359e+00]
0 olmayan özniteliklerin sayısı: 6
R-kare eğitim: 0.693
R-kare test: 0.600
0 olmayan özniteliklerin ağırlıkları(mutlak değer sırasında):
	LSTAT, -3.404
	RM, 2.675
	PTRATIO, -1.785
	TAX, -0.120
	CRIM, -0.034
	B, 0.002


In [10]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 2) #Polinom derecesi arttıkça modelin karmaşıklık derecesi arttığı için aşırı öğrenmeye yol açar.
X_poly = poly.fit_transform(X)
X_train, X_test, y_train1, y_test1 = train_test_split(X_poly, Y, random_state = 0)
#Normalize edilmiş veri ile çalışılır.(std)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
linreg1 = LinearRegression().fit(X_train_std, y_train1)

print("(polinom model)katsayılar (w):\n{}" .format(linreg1.coef_))
print("(polinom model) sabit (b): {:.3f}" .format(linreg1.intercept_))
print("(polinom model) R-kare eğitim: {:.3f}" .format(linreg1.score(X_train_std, y_train1.values.ravel())))
print("(polinom model) R-kare test: {:.3f}\n" .format(linreg1.score(X_test_std, y_test1.values.ravel())))
print("0 olmayan öznitelik(katsayı) sayısı: {}" .format(np.sum(linreg1.coef_ != 0)))

pre_train = linreg1.predict(X_train_std)
pre_test = linreg1.predict(X_test_std)
print("MSE-eğitim: %.3f" % mean_squared_error(y_train1, pre_train))
print("MSE-test: %.3f" % mean_squared_error(y_test1, pre_test))
print("R-kare (eğitim): %.4f" % r2_score(y_train1, pre_train))
print("R-kare (test): %.4f" % r2_score(y_test1, pre_test))

(polinom model)katsayılar (w):
[-3.17763383e-11 -1.36066033e+00  2.32738250e+00 -4.21315711e+01
  4.29172126e+00 -3.20474118e+00  3.18525454e+00  3.48051577e+01
 -2.23377269e+01  9.73693788e+00  1.08958586e+01  5.42661130e+00
 -1.48379531e+01 -1.08854941e+01  7.76299031e-01  8.31306070e-01
  9.26243771e+01  9.82253830e-01 -2.20714474e+00  4.43224885e+00
 -2.19208802e+00  7.72268880e-01  1.24671161e+02 -2.65145003e+02
  4.44401004e+01 -1.76290367e-01  2.58194791e+00 -2.53470412e+00
 -6.03828745e-01 -1.01738819e-01 -1.65683922e+01  3.02487453e+00
  4.03107235e-01 -1.30655611e-01 -3.13183382e-01  4.60514099e+00
 -4.88433618e+00  1.59704548e+01 -7.28503539e-01  1.01652955e+01
  1.41666968e-01 -2.98879249e-01  1.67823036e+01  3.99652050e+00
  2.54412929e+00 -8.95841966e+00  6.54809391e+00 -7.61415902e+00
  1.47567065e+01 -2.10305318e+00  4.29172126e+00 -5.41588829e+00
 -1.06575083e+01  1.43600773e+00 -6.90466770e-01  2.38250991e+00
 -2.55505809e+00 -5.63746082e+00  1.38061612e+01 -1.6208617

In [21]:
# 2- K-Nearest Neighbor Regresyon
from sklearn.neighbors import KNeighborsRegressor
kNN = KNeighborsRegressor(n_neighbors = 10)#Karar değişkeni-parametre n_neighbors(komşu sayısı)dır.Sayısı düşük iken aşırı öğrenme olur.
kNN.fit(x_train_std, y_train)
#Normalize edilmiş veri ile çalışılır.(std)

pred_train = kNN.predict(x_train_std)
pred_test = kNN.predict(x_test_std)
print("R-kare eğitim: %.4f" % r2_score(y_train, pred_train))
print("R-kare test: %.4f" % r2_score(y_test, pred_test))
print("MSE-eğitim: %.3f" % mean_squared_error(y_train,pred_train))
print("MSE-test: %.3f" % mean_squared_error(y_test, pred_test))

R-kare eğitim: 0.7996
R-kare test: 0.6563
MSE-eğitim: 16.985
MSE-test: 28.621


In [29]:
# 3- Karar Ağaçları
from sklearn.tree import DecisionTreeRegressor

KAR = DecisionTreeRegressor(max_leaf_nodes = 20, random_state = 0, max_depth = 6)
KAR.fit(x_train, y_train)

pred_train = KAR.predict(x_train)
pred_test = KAR.predict(x_test)

print("R-kare eğitim: %.4f" % r2_score(y_train, pred_train))
print("R-kare test: %.4f" % r2_score(y_test, pred_test))
print("MSE-eğitim: %.3f" % mean_squared_error(y_train,pred_train))
print("MSE-test: %.3f" % mean_squared_error(y_test, pred_test))

R-kare eğitim: 0.9274
R-kare test: 0.7120
MSE-eğitim: 6.156
MSE-test: 23.981


In [33]:
# 4- Yapay Sinir Ağları (Multilayer Perceptron - neural network)
from sklearn.neural_network import MLPRegressor
#Normalize edilmiş veri ile çalışılır. (std)
# Aktivasyon fonksiyonları--> activation : {"identity", "logistic", "tanh", "relu"}
mlp = MLPRegressor(activation = "relu", hidden_layer_sizes = [10], solver = "lbfgs",max_iter = 20000, tol = 0.000000001 )
mlp.fit(x_train_std, y_train)

pre_train = mlp.predict(x_train_std)
pre_test = mlp.predict(x_test_std)

print("R-kare eğitim: %.4f" % r2_score(y_train, pre_train))
print("R-kare test: %.4f" % r2_score(y_test, pre_test))
print("MSE-eğitim: %.3f" % mean_squared_error(y_train,pre_train))
print("MSE-test: %.3f" % mean_squared_error(y_test, pre_test))

R-kare eğitim: 0.9670
R-kare test: 0.7630
MSE-eğitim: 2.795
MSE-test: 19.738


In [52]:
# 5- XGBOOST
import xgboost

xgb = xgboost.XGBRegressor(objective = "reg:squarederror", n_estimators =300, learning_rate = 0.08, max_depth = 10)
xgb.fit(x_train, y_train)

pre_train = xgb.predict(x_train)
pre_test = xgb.predict(x_test)

print("R-kare eğitim: %.4f" % r2_score(y_train, pre_train))
print("R-kare test: %.4f" % r2_score(y_test, pre_test))
print("MSE-eğitim: %.3f" % mean_squared_error(y_train,pre_train))
print("MSE-test: %.3f" % mean_squared_error(y_test, pre_test))

C:\Users\batuh\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


R-kare eğitim: 1.0000
R-kare test: 0.7739
MSE-eğitim: 0.000
MSE-test: 18.830


In [55]:
# 6- RASTGELE ORMAN ALGORİTMASI
from sklearn.ensemble import RandomForestRegressor
rf_clf = RandomForestRegressor(max_depth = 15, n_estimators = 100, min_samples_leaf = 10, min_samples_split = 10, random_state = 10)
rf_clf.fit(x_train, y_train)

pre_train = rf_clf.predict(x_train)
pre_test = rf_clf.predict(x_test)

print("R-kare eğitim: %.4f" % r2_score(y_train, pre_train))
print("R-kare test: %.4f" % r2_score(y_test, pre_test))
print("MSE-eğitim: %.3f" % mean_squared_error(y_train,pre_train))
print("MSE-test: %.3f" % mean_squared_error(y_test, pre_test))

R-kare eğitim: 0.8848
R-kare test: 0.7559
MSE-eğitim: 9.765
MSE-test: 20.324


In [57]:
# PARAMETRE OPTİMİZASYONU
# 1.Ridge regresyon (L2) ve düzenlileştirme katsayısı
print("Ridge regresyon: düzenlileştirme katsayısının etkisi\n")
for alfa in [0.05, 0.1, 0, 1, 10, 20, 50, 100, 1000]:
    linridge = Ridge(alpha = alfa).fit(x_train_std, y_train)
    print("Lambda = {:.2f}\n" .format(alfa))
    pre_train = linridge.predict(x_train_std)
    pre_test = linridge.predict(x_test_std)
    print("R-Kare (eğitim): %.3f" % r2_score(y_train, pre_train))
    print("R-Kare (test): %.3f \n" % r2_score(y_test, pre_test))

Ridge regresyon: düzenlileştirme katsayısının etkisi

Lambda = 0.05

R-Kare (eğitim): 0.765
R-Kare (test): 0.673 

Lambda = 0.10

R-Kare (eğitim): 0.765
R-Kare (test): 0.673 

Lambda = 0.00

R-Kare (eğitim): 0.765
R-Kare (test): 0.673 

Lambda = 1.00

R-Kare (eğitim): 0.765
R-Kare (test): 0.673 

Lambda = 10.00

R-Kare (eğitim): 0.763
R-Kare (test): 0.667 

Lambda = 20.00

R-Kare (eğitim): 0.761
R-Kare (test): 0.662 

Lambda = 50.00

R-Kare (eğitim): 0.753
R-Kare (test): 0.649 

Lambda = 100.00

R-Kare (eğitim): 0.739
R-Kare (test): 0.632 

Lambda = 1000.00

R-Kare (eğitim): 0.537
R-Kare (test): 0.461 



In [64]:
# 2. Lasso Regresyon (L1) ve düzenlileştirme katsayısı
print("Lasso regresyon: düzenlileştirme katsayısının etkisi\n")
for alfa in [0.001, 0.01, 0.5, 1, 2, 3, 5, 10, 20, 50]:
    linlasso = Lasso(alpha = alfa, max_iter = 150000).fit(x_train_std, y_train)
    print("Lambda = {:.3f}\n" .format(alfa))
    pre_train = linlasso.predict(x_train_std)
    pre_test = linlasso.predict(x_test_std)
    print("R-Kare (eğitim): %.3f" % r2_score(y_train, pre_train))
    print("R-Kare (test): %.3f" % r2_score(y_test, pre_test))
    print("0 olmayan öznitelik sayısı: {}\n" .format(np.sum(linlasso.coef_ != 0)))

Lasso regresyon: düzenlileştirme katsayısının etkisi

Lambda = 0.001

R-Kare (eğitim): 0.765
R-Kare (test): 0.673
0 olmayan öznitelikler: 13

Lambda = 0.010

R-Kare (eğitim): 0.764
R-Kare (test): 0.672
0 olmayan öznitelikler: 12

Lambda = 0.500

R-Kare (eğitim): 0.720
R-Kare (test): 0.618
0 olmayan öznitelikler: 8

Lambda = 1.000

R-Kare (eğitim): 0.693
R-Kare (test): 0.600
0 olmayan öznitelikler: 6

Lambda = 2.000

R-Kare (eğitim): 0.636
R-Kare (test): 0.568
0 olmayan öznitelikler: 3

Lambda = 3.000

R-Kare (eğitim): 0.543
R-Kare (test): 0.503
0 olmayan öznitelikler: 3

Lambda = 5.000

R-Kare (eğitim): 0.288
R-Kare (test): 0.280
0 olmayan öznitelikler: 2

Lambda = 10.000

R-Kare (eğitim): 0.000
R-Kare (test): -0.006
0 olmayan öznitelikler: 0

Lambda = 20.000

R-Kare (eğitim): 0.000
R-Kare (test): -0.006
0 olmayan öznitelikler: 0

Lambda = 50.000

R-Kare (eğitim): 0.000
R-Kare (test): -0.006
0 olmayan öznitelikler: 0



In [65]:
# 3. Polinom Derecesi
for a in range (1,7): # Derece 1 = Doğrusal Regresyon
    tic = time.time()
    tahminpolinom = PolynomialFeatures(degree = a)
    Xyeni = tahminpolinom.fit_transform(X)
    X_train1, X_test1, y_train1, y_test1 = train_test_split(Xyeni, Y, test_size = 0.3, random_state = 0)
    sc = StandardScaler()
    sc.fit(X_train1)
    x_train_st1 = sc.transform(X_train1)
    x_test_st1 = sc.transform(X_test1)
    
    polimodel = LinearRegression()
    polimodel.fit(x_train_st1, y_train1)
    pre_train = polimodel.predict(x_train_st1)
    pre_test = polimodel.predict(x_test_st1)
    print("Derece = ", a)
    print("\n")
    print("MSE-eğitim: %.3f" % mean_squared_error(y_train1, pre_train))
    print("MSE-test: %.3f" % mean_squared_error(y_test1, pre_test))
    print("R-Kare (eğitim): %.3f" % r2_score(y_train1, pre_train))
    print("R-Kare (test): %.3f" % r2_score(y_test1, pre_test))
    print("0 olmayan öznitelik sayısı: {}\n" .format(np.sum(polimodel.coef_ != 0)))
    toc = time.time()
    print(toc-tic, "sn geçti\n")

Derece =  1


MSE-eğitim: 19.958
MSE-test: 27.196
R-Kare (eğitim): 0.765
R-Kare (test): 0.673
0 olmayan öznitelik sayısı: 13

0.01698899269104004 sn geçti

Derece =  2


MSE-eğitim: 4.117
MSE-test: 29.550
R-Kare (eğitim): 0.951
R-Kare (test): 0.645
0 olmayan öznitelik sayısı: 105

0.02698349952697754 sn geçti

Derece =  3


MSE-eğitim: 0.000
MSE-test: 19354.182
R-Kare (eğitim): 1.000
R-Kare (test): -231.439
0 olmayan öznitelik sayısı: 560

0.20487332344055176 sn geçti

Derece =  4


MSE-eğitim: 0.000
MSE-test: 2714.607
R-Kare (eğitim): 1.000
R-Kare (test): -31.602
0 olmayan öznitelik sayısı: 2380

0.4740936756134033 sn geçti

Derece =  5


MSE-eğitim: 0.000
MSE-test: 2146.488
R-Kare (eğitim): 1.000
R-Kare (test): -24.779
0 olmayan öznitelik sayısı: 8568

0.8794569969177246 sn geçti

Derece =  6


MSE-eğitim: 0.000
MSE-test: 1683.270
R-Kare (eğitim): 1.000
R-Kare (test): -19.216
0 olmayan öznitelik sayısı: 27132

2.6693432331085205 sn geçti



In [66]:
# Polinom + Lasso
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 6)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train1, y_test1 = train_test_split(X_poly, Y, random_state = 0)

sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

linlasso1 = Lasso(max_iter = 20000).fit(X_train_std, y_train1)

print("Polinom model katsayılar (w):\n{} " .format(linlasso1.coef_))
print("Polinom model sabit (b):\n{:.3f} " .format(linlasso1.intercept_))
print("Polinom model R-Kare(eğitim): {:.3f}" .format(linlasso1.score(X_train_std, y_train1.values.ravel())))
print("Polinom model R-Kare(test): {:.3f}" .format(linlasso1.score(X_test_std, y_test1.values.ravel())))
print("0 olmayan öznitelik sayısı: {}\n" .format(np.sum(linlasso1.coef_ != 0)))

pre_train = linlasso1.predict(X_train_std)
pre_test = linlasso1.predict(X_test_std)
print("MSE-eğitim: %.3f" % mean_squared_error(y_train1, pre_train))
print("MSE-test: %.3f" % mean_squared_error(y_test1, pre_test))
print("R-Kare eğitim: %.4f" % r2_score(y_train1, pre_train))
print("R-Kare test: %.4f" % r2_score(y_test1, pre_test))

Polinom model katsayılar (w):
[ 0. -0.  0. ... -0. -0. -0.] 
Polinom model sabit (b):
22.609 
Polinom model R-Kare(eğitim): 0.796
Polinom model R-Kare(test): 0.686
0 olmayan öznitelik sayısı: 12

MSE-eğitim: 17.411
MSE-test: 25.613
R-Kare eğitim: 0.7959
R-Kare test: 0.6865
